In [ ]:
import numpy as np
import arviz as az
import pymc as pm

In [ ]:
# a)

publicity = np.array([1.5, 2.0, 2.3, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0,
                      6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0])
sales = np.array([5.2, 6.8, 7.5, 8.0, 9.0, 10.2, 11.5, 12.0, 13.5, 14.0,
                  15.0, 15.5, 16.2, 17.0, 18.0, 18.5, 19.5, 20.0, 21.0, 22.0])

new_publicity = np.array([6.0, 8.0, 10.0])

with pm.Model() as model:
    x = pm.MutableData("publicity", publicity)

    alpha = pm.Normal("alpha", mu=0, sigma=20)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=5)

    mu = alpha + beta * x
    pm.Normal("obs", mu=mu, sigma=sigma, observed=sales)

    idata = pm.sample(2000, tune=1000, target_accept=0.9, random_seed=1)

    alpha_est = idata.posterior["alpha"].mean().item()
    beta_est  = idata.posterior["beta"].mean().item()

    print("alpha estimate:", alpha_est)
    print("beta estimate:", beta_est)


In [ ]:
# b)

coef_hdi_94 = az.hdi(idata, var_names=["alpha", "beta"], hdi_prob=0.94)
print("94% HDI:\n", coef_hdi_94)

In [ ]:
# c)

with model:
    pm.set_data({"publicity": new_publicity})
    ppc = pm.sample_posterior_predictive(idata, var_names=["obs"], random_seed=1)

samples = ppc.posterior_predictive["obs"].stack(sample=("chain","draw")).values
for i, xval in enumerate(new_publicity):
    mean_pred = samples[:, i].mean()
    lo, hi = np.percentile(samples[:, i], [5, 95])
    print(f"publicity={xval} -> mean={mean_pred:.3f}, 90% PI=[{lo:.3f}, {hi:.3f}]")